In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.decomposition import PCA

from itertools import cycle, islice

In [2]:
ed_data = pd.read_csv('../edviz_raw.csv')

In [78]:
y = ed_data[['disposition', 'esi']]
X = ed_data.drop(columns=['disposition', 'esi'])

In [79]:
unnorm_X = X.copy()

X_cat = X.select_dtypes(exclude=['int64', 'float64'])
X_dog = X.select_dtypes(include=['int64', 'float64'])

X_cat = pd.get_dummies(X_cat)

X = pd.concat([X_cat, X_dog], axis=1)

X = (X - X.min())/(X.max()-X.min())
#X = (X - X.mean()/X.std())

X = X.fillna(X.mean())

X.dropna(axis=1, how='all', inplace=True)

In [80]:
pca = PCA(n_components=10)
pca.fit(norm_X)
X_trans = pca.transform(norm_X)

y_true = pd.get_dummies(y['disposition'])['Admit']

In [81]:
X_large = X.copy()
X_large.dropna(axis=1, how='all', inplace=True)
X_large[['PCA_0', 'PCA_1']] = pd.DataFrame(X_trans)[[0,1]]

In [108]:
def describe_peak(df, df_embedding, peak, radius=0.05, verbose=False):
    
    dim = len(peak)
    df_embedding = df_embedding[:,0:dim]
    
    peak_entries = df[((df_embedding - peak)**2).sum(axis=1) < radius**2].describe().T
    
    if verbose:
        df_stats = df.describe().T
        print('There are %d samples (%f%%) in this peak.'
              % (peak_entries.iloc[0]['count'],
                 peak_entries.iloc[0]['count']/df_stats.iloc[0]['count'] * 100))
        diff = (peak_entries - df_stats)
        print('This sample is notable for these features above the mean:')
        display(diff['mean'].sort_values(ascending=False).head(10))
        print('This sample is notable for these features below the mean:')
        display(diff['mean'].sort_values(ascending=True).head(10))
    
    else:
        return peak_entries

In [109]:
describe_peak(X, X_trans, (.2,2), radius=.05, verbose=True)

There are 44 samples (0.007850%) in this peak.
This sample is notable for these features above the mean:


asthma                            0.582271
insurance_status_Medicaid         0.561519
otjointdx                         0.552127
htn                               0.544627
esophgealdx                       0.518342
unclassified                      0.515165
diabmelnoc                        0.491246
backproblem                       0.449372
race_Black or African American    0.445581
copd                              0.443612
Name: mean, dtype: float64

This sample is notable for these features below the mean:


race_White or Caucasian           -0.511866
arrivalmode_Car                   -0.398022
insurance_status_Commercial       -0.316188
previousdispo_No previous dispo   -0.292628
gender_Male                       -0.265709
maritalstatus_Married             -0.249500
religion_Catholic                 -0.246261
employstatus_Full Time            -0.243813
dep_name_A                        -0.234097
employstatus_Retired              -0.232068
Name: mean, dtype: float64